## Transfer Learning using `ResNet50` on `CIFAR-10` imageset.
### 1. Importing libraries

In [ ]:
import tensorflow as tf 
from tensorflow.keras.applications.resnet50 import ResNet50

### 2. Load data & preprocess.

In [15]:
(train_image, train_label),(val_image, val_label) = tf.keras.datasets.cifar10.load_data()

In [16]:
def preprocess_img(input_images):
    input_images = input_images.astype('float32')
    output_images = tf.keras.applications.resnet50.preprocess_input(input_images)
    return output_images
train_x = preprocess_img(train_image)
val_x = preprocess_img(val_image)

### 3. Build and Compile the Model.

In [17]:
def construct_model():
    inputs = tf.keras.layers.Input(shape=(32,32,3))
    resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)
    feature_extractor = ResNet50(input_shape=(224,224,3), include_top=False, weights='imagenet')(resize)
    x = tf.keras.layers.GlobalAveragePooling2D()(feature_extractor)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dense(512, activation='relu', name='classification')(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model
model = construct_model()
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
classification (Dense)       (None, 512)               5248

### 4. Model Train

- As it consumes a lot of time, i only trained for 1 epoch.Feel free to change.

In [19]:
history = model.fit(train_x, train_label, epochs=1, validation_data=(val_x, val_label), batch_size=64)

782/782 [==============================] - 37496s 48s/step - loss: 0.7410 - accuracy: 0.7564 - val_loss: 0.3683 - val_accuracy: 0.8787


### Visualize Loss and Accuracy
- As i have trained for only one epoch, plotting wouldn't make sense for this scenario.

In [24]:
import matplotlib.pyplot as plt
def plot_metrics(metric_name, title, ylim=5):
    plt.title(title)
    plt.ylim(0, ylim)
    plt.plot(history.history[metric_name], color='blue', label=metric_name)
    plt.plot(history.history['val_'+metric_name], color='red', label='val'+metric_name)